In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from time import sleep
from datetime import datetime
from splinter import Browser
from selenium.webdriver.common.keys import Keys
import time
import json

In [2]:
# Path to chromedriver.exe
executable_path = {'executable_path':'C:/Users/zacha/AppData/Local/bin/chromedriver.exe'}

# Open the Splinter Browswer
browser = Browser('chrome', **executable_path)

In [3]:
# URL to find information on Coronavirus
url = "https://txdshs.maps.arcgis.com/apps/opsdashboard/index.html#/ed483ecd702b4298ab01e8b9cafc8b83"

url3 = "http://publichealth.harriscountytx.gov/Resources/2019-Novel-Coronavirus/Harris-County-COVID-19-Confirmed-Cases"

In [4]:
# Visiting the url 
browser.visit(url)

In [5]:
time.sleep(5)# Creating the Beautiful soup for URL
html = browser.html
soup = bs(html, 'html.parser')

In [6]:
# All the Texas counties
counties = soup.find('nav', attrs={'class':'feature-list'})

# getting all the spans with the information for each Texas county
rows = counties.find_all('span')




In [7]:
counter = 0

county = []
cases = []

for row in rows:
    try:
        print(rows[counter].text.strip().split('\n'))
        county.append(rows[counter].text.strip().split('\n')[0])
        cases.append(rows[counter].text.strip().split('\n')[1])
        counter = counter + 3
        
    except:
        print('failure')

['Dallas County', '303']
['Harris County', '185']
['Travis County', '119']
['Tarrant County', '90']
['Bexar County', '84']
['Denton County', '70']
['Fort Bend County', '54']
['Collin County', '53']
['Brazoria County', '33']
['Montgomery County', '31']
['McLennan County', '28']
['Galveston County', '27']
['Williamson County', '22']
['Brazos County', '21']
['El Paso County', '21']
['Bell County', '19']
['Lubbock County', '19']
['Smith County', '16']
['Nueces County', '14']
['Webb County', '10']
['Hays County', '9']
['Matagorda County', '9']
['Guadalupe County', '8']
['Wichita County', '8']
['Cameron County', '7']
['Comal County', '7']
['Hidalgo County', '7']
['Jefferson County', '7']
['Castro County', '6']
['Ellis County', '6']
['Hockley County', '5']
['Midland County', '4']
['Brown County', '3']
['Gregg County', '3']
['Johnson County', '3']
['Kendall County', '3']
['Terry County', '3']
['Victoria County', '3']
['Walker County', '3']
['Wharton County', '3']
['Atascosa County', '2']
['Bas

In [8]:
df = pd.DataFrame(list(zip(county, cases)), columns =['Texas_County', 'COVID_19_Cases'])
df

,Texas_County,COVID_19_Cases
0,Dallas County,303
1,Harris County,185
2,Travis County,119
3,Tarrant County,90
4,Bexar County,84
...,...,...
87,San Patricio County,1
88,Upshur County,1
89,Val Verde County,1
90,Van Zandt County,1


In [9]:
# Droping the word 'County'
df['Texas_County'] = df['Texas_County'].apply(lambda x : x.split(' County')[0])

In [10]:
# Revise? 
reported_cases = soup.find_all('text')[6].text
reported_cases

'1,396'

In [11]:
deaths = soup.find_all('text')[8].text
deaths

'18'

In [12]:
df_stats = pd.DataFrame({'Cases':[reported_cases], 'Deaths':[deaths]})
df_stats.to_csv('static/csv/stats.csv', index=False)

In [13]:
browser.visit(url3)

In [14]:
harris_table = browser.find_by_id('covidtable')

In [15]:
html_harris = browser.html
harris_soup = bs(html_harris, 'html.parser')

In [16]:
harris_data = []
table = harris_soup.find('table', attrs={'id':'covidtable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    harris_data.append([ele for ele in cols if ele]) # Get rid of empty values

In [17]:
columns = ['Sex', 'Location', 'Age_Range', 'Status']


In [18]:
harris_df = pd.DataFrame(harris_data, columns=columns)

In [19]:
harris_df['Sex'] = harris_df['Sex'].apply(lambda x : x.split()[2])

In [20]:
harris_df.to_csv('static/csv/harris_stats.csv', index=False, header=True)

In [21]:
harris_df.head()

,Sex,Location,Age_Range,Status
0,Female,SW,60-69,Recovered
1,Male,SW,60-69,Recovered
2,Male,NW,60-69,Recovered
3,Female,NW,60-69,Recovered
4,Female,SW,20-29,Recovered


In [22]:
All_Tex_Counties_df = pd.read_csv('static/csv/Texas_Counties.csv')
All_Tex_Counties_df.head()

,X (Lat),Y (Long),CNTY_NM,CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,-97.492799,29.456415,Gonzales,90,48177,2.124911,0.257805,"(-97.492799, 29.456415)"
1,-98.697292,27.043405,Jim Hogg,125,48247,2.271751,0.267624,"(-98.697292, 27.043405)"
2,-97.681378,26.924094,Kenedy,66,48261,5.067864,0.389397,"(-97.681378, 26.924094)"
3,-96.965687,30.310651,Lee,144,48287,2.213853,0.153990,"(-96.965687, 30.310651)"
4,-95.853568,32.211881,Henderson,108,48213,3.152909,0.235056,"(-95.853568, 32.211881)"


In [23]:
All_Tex_Counties = All_Tex_Counties_df['CNTY_NM'].tolist()

In [24]:
df_counties = list(df['Texas_County'].apply(lambda x : x.split()[0]))

In [25]:
Counties_with_Cases = []

for i in df_counties:
    
    if i in All_Tex_Counties:
        Counties_with_Cases.append(i)
        print(f'{i}---found')
        
    elif i == 'DeWitt':
        Counties_with_Cases.append('De Witt')
        print(f'{i}---found')
    else:
        print(f'Houston we have a problem: {i}')

Dallas---found
Harris---found
Travis---found
Tarrant---found
Bexar---found
Denton---found
Houston we have a problem: Fort
Collin---found
Brazoria---found
Montgomery---found
McLennan---found
Galveston---found
Williamson---found
Brazos---found
Houston we have a problem: El
Bell---found
Lubbock---found
Smith---found
Nueces---found
Webb---found
Hays---found
Matagorda---found
Guadalupe---found
Wichita---found
Cameron---found
Comal---found
Hidalgo---found
Jefferson---found
Castro---found
Ellis---found
Hockley---found
Midland---found
Brown---found
Gregg---found
Johnson---found
Kendall---found
Terry---found
Victoria---found
Walker---found
Wharton---found
Atascosa---found
Bastrop---found
Calhoun---found
Chambers---found
Houston we have a problem: Deaf
Houston we have a problem: De
Eastland---found
Grayson---found
Grimes---found
Hardin---found
Kaufman---found
Llano---found
Milam---found
Parker---found
Potter---found
Rockwall---found
Rusk---found
Houston we have a problem: Tom
Angelina---found
Au

In [26]:
All_Tex_Counties_df = All_Tex_Counties_df.set_index('CNTY_NM')

In [27]:
Geo_df = All_Tex_Counties_df.loc[Counties_with_Cases]
Geo_df.head()

,X (Lat),Y (Long),CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
CNTY_NM,,,,,,,
Dallas,-96.777819,32.766537,57,48113,1.907782,0.226421,"(-96.777819, 32.766537)"
Harris,-95.397821,29.859671,102,48201,7.731110,0.422798,"(-95.397821, 29.859671)"
Travis,-97.781947,30.334233,227,48453,2.458272,0.248996,"(-97.781947, 30.334233)"
Tarrant,-97.291165,32.771852,220,48439,1.905740,0.225294,"(-97.291165, 32.771852)"
Bexar,-98.519663,29.448845,15,48029,2.729791,0.302846,"(-98.519663, 29.448845)"


In [28]:
Geo_df = Geo_df.reset_index()
Geo_df = Geo_df.rename(columns={"CNTY_NM": "Texas_County"})
Geo_df

,Texas_County,X (Lat),Y (Long),CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,Dallas,-96.777819,32.766537,57,48113,1.907782,0.226421,"(-96.777819, 32.766537)"
1,Harris,-95.397821,29.859671,102,48201,7.731110,0.422798,"(-95.397821, 29.859671)"
2,Travis,-97.781947,30.334233,227,48453,2.458272,0.248996,"(-97.781947, 30.334233)"
3,Tarrant,-97.291165,32.771852,220,48439,1.905740,0.225294,"(-97.291165, 32.771852)"
4,Bexar,-98.519663,29.448845,15,48029,2.729791,0.302846,"(-98.519663, 29.448845)"
...,...,...,...,...,...,...,...,...
79,Oldham,-102.602995,35.404976,180,48359,2.638664,0.385883,"(-102.602995, 35.404976)"
80,Orange,-93.896538,30.128071,181,48361,1.872030,0.088532,"(-93.896538, 30.128071)"
81,Robertson,-96.512726,31.027054,198,48395,2.503123,0.211742,"(-96.512726, 31.027054)"
82,Upshur,-94.941320,32.736283,230,48459,1.799823,0.147655,"(-94.94132, 32.736283)"


In [29]:
result = pd.merge(df, Geo_df, on='Texas_County')
result = result.rename(columns={'X (Lat)':'long', 'Y (Long)':'lat'})

result.head()

,Texas_County,COVID_19_Cases,long,lat,CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,Dallas,303,-96.777819,32.766537,57,48113,1.907782,0.226421,"(-96.777819, 32.766537)"
1,Harris,185,-95.397821,29.859671,102,48201,7.731110,0.422798,"(-95.397821, 29.859671)"
2,Travis,119,-97.781947,30.334233,227,48453,2.458272,0.248996,"(-97.781947, 30.334233)"
3,Tarrant,90,-97.291165,32.771852,220,48439,1.905740,0.225294,"(-97.291165, 32.771852)"
4,Bexar,84,-98.519663,29.448845,15,48029,2.729791,0.302846,"(-98.519663, 29.448845)"


In [30]:
result = result[['Texas_County', 'COVID_19_Cases', 'lat', 'long']]
result

,Texas_County,COVID_19_Cases,lat,long
0,Dallas,303,32.766537,-96.777819
1,Harris,185,29.859671,-95.397821
2,Travis,119,30.334233,-97.781947
3,Tarrant,90,32.771852,-97.291165
4,Bexar,84,29.448845,-98.519663
...,...,...,...,...
79,Oldham,1,35.404976,-102.602995
80,Orange,1,30.128071,-93.896538
81,Robertson,1,31.027054,-96.512726
82,Upshur,1,32.736283,-94.941320


In [31]:
# # Adding lat and longs to the dataframe
# df['lat'] = lats
# df['long'] = longs

In [32]:
# df.head()

In [33]:
browser.quit()

In [34]:
def df_to_geojson(df, properties, lat='lat', lon='long'):
    prefix = 'var' + ' data' + ' = '
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point','coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lat],row[lon]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

cols = ['Texas_County', 'COVID_19_Cases']
javascript_file = df_to_geojson(result, cols)
# {'type': 'FeatureCollection', 'features': [{'geometry': {'type': 'Point', 'coordinates': [-70.5867, -29.9953, 760]}, 'type': 'Feature', 'properties': {'name': 'A', 'description': 'Place a'}}, {'geometry': {'type': 'Point', 'coordinates': [-70.4933, -30.1217, 1250]}, 'type': 'Feature', 'properties': {'name': 'B', 'description': 'Place b'}}, {'geometry': {'type': 'Point', 'coordinates': [-70.5008, -30.0953, 1185]}, 'type': 'Feature', 'properties': {'name': 'C', 'description': 'Place c'}}]}

In [35]:

with open('static/js/map.json','w') as f:
    json.dump(javascript_file, f, indent=4, sort_keys=False)